In [166]:
import os
import ast
import json
import importlib 
import itertools
import collections
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend, ColumnDataSource
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr


import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw


from IPython import display

importlib.reload(bestfits)
importlib.reload(setup_bestfit)
importlib.reload(setup_overlap)
importlib.reload(pipe_lblnew)
importlib.reload(setup_cliradlw)
importlib.reload(pipe_cliradlw)

<module 'climatools.cliradlw.pipeline' from '/chia_cluster/home/jackyu/climatools/climatools/cliradlw/pipeline.py'>

In [167]:
output_notebook()

Loading BokehJS ...

In [168]:
'''
Get the clirad-lw and lblnew `param`s for all spectral bands.  
These are returned by functions `clirad_params_atm` and
`lblnew_params_atm`, respectively.
'''

def molecules_byband_atm():
    return {1: {'h2o': 'atmpro'},
            2: {'h2o': 'atmpro'}, 
            3: {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07},
            4: {'co2': 0.0004, 'h2o': 'atmpro'},
            5: {'co2': 0.0004, 'h2o': 'atmpro'},
            6: {'co2': 0.0004, 'h2o': 'atmpro'},
            7: {'co2': 0.0004, 'h2o': 'atmpro', 'o3': 'atmpro'},
            8: {'h2o': 'atmpro'},
            9: {'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 3.2e-07},
            10: {'h2o': 'atmpro'},
            11: {'co2': 0.0004, 'h2o': 'atmpro'}}


def clirad_params_atm():
    d = {}
    for band, molecule in molecules_byband_atm().items():
        for param in setup_cliradlw.test_cases():
            if [band] == param['band'] and molecule == param['molecule']:
                d[band] = param
                break                
    return d


def analysis_dirs_atm():
    params = clirad_params_atm()
    return {band: pipe_cliradlw.get_analysis_dir(param=param,
                                                 setup=setup_cliradlw) 
            for band, param in params.items()}


def lblnew_params_atm():
    dirs = analysis_dirs_atm()
    
    d = {}
    for band, dirname in dirs.items():
        with open(os.path.join(dirname, 'param.py'), 
                  mode='r', encoding='utf-8') as f:
            _, l = f.readlines()
        
        s = l.split('=')[1].strip()
        d[band] = ast.literal_eval(s)
        
    return d

    


In [169]:
def script_atm_makeup():
    df = pd.DataFrame()

    for band, molecule in molecules_byband_atm().items():
        for name, conc in molecule.items():
            df.loc[name, band] = str(conc)

    df = df.fillna(0)
    df.columns.name = 'clirad band'
    df.index.name = 'molecule'
    
    display.display(df)
    
    print('Table. The makeup of the atmosphere.')



In [170]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [171]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict

In [172]:
def crd_data_atm(params_atm):
    '''
    Parameters
    ----------
    
    params_atm: dict
                Parameter dictionary for each band.
                
    d: dict
       'flux': xr.Dataset. [pressure, band]
               Fluxes.
       'cool': xr.Dataset. [pressure, band]
               Cooling rate.
    
    '''
    
    results_atm = {band: load_lblnew_data(param) 
                   for band, param in params_atm.items()}
    
    bands = [band for band, _ in params_atm.items()]
    fluxs = [d['ds_flux_crd'] for _, d in results_atm.items()]
    cools = [d['ds_cool_crd'] for _, d in results_atm.items()]
    
    d = {}
    d['flux'] = xr.concat(fluxs, dim=bands).rename({'concat_dim': 'band'})
    d['cool'] = xr.concat(cools, dim=bands).rename({'concat_dim': 'band'})
    return d        
        
        

def clirad_data_atm(params_atm):
    
    dirnames = [pipe_cliradlw.get_fortran_dir(param,
                                              setup=setup_cliradlw)
                for _, param in params_atm.items()]
    
    fpaths_flux = [os.path.join(n, 'output_flux.dat') for n in dirnames]
    fpaths_cool = [os.path.join(n, 'output_coolr.dat') for n in dirnames]
    
    bands = [band for band, _ in params_atm.items()]
    fluxs = [load_output_file(p) for p in fpaths_flux]    
    cools = [load_output_file(p) for p in fpaths_cool]
    
    d = {}
    d['flux'] = sum(fluxs).sel(band=bands)
    d['cool'] = sum(cools).sel(band=bands)
    return d



In [173]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']



def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)



def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))

In [174]:
def pltdata_cool():
    
    d_clirad = clirad_data_atm(clirad_params_atm())
    d_crd = crd_data_atm(lblnew_params_atm())
    
    ds_clirad = d_clirad['cool']
    ds_crd = d_crd['cool']
    
    colors = all_palettes['Set1'][4]
    
    data = [
        {'label': 'CLIRAD',
         'srs': fmt_cool(ds_clirad),
         'line_dash': 'dashed', 'line_width': 5,
         'color': colors[0], 'alpha': .6},
        {'label': 'CRD',
         'srs': fmt_cool(ds_crd),
         'line_dash': 'solid', 'line_width': 1.5,
         'marker': 'circle', 'marker_size': 5,
         'color': colors[2], 'alpha': 1}
    ]
    return data


def pltdata_cooldiff():
    
    d_clirad = clirad_data_atm(clirad_params_atm())
    d_crd = crd_data_atm(lblnew_params_atm())
    
    ds_clirad = d_clirad['cool']
    ds_crd = d_crd['cool']
    
    ds_diff = ds_clirad - ds_crd
    
    colors = all_palettes['Set1'][4]
    
    data = [
        {'label': 'CLIRAD - CRD',
         'srs': fmt_cool(ds_diff),
         'line_dash': 'solid', 'line_width': 1.5, 
         'marker': 'circle', 'marker_size': 7,
         'color': colors[3], 'alpha': .8}
    ]
    return data


def script_cool():
    plt_cooling_bokeh(pltdata=pltdata_cool())
    print('Figure. Cooling rate.')
    print()
    print()
    print()
    plt_cooling_bokeh(pltdata=pltdata_cooldiff())
    print('Figure. Cooling rate difference.')
    

In [175]:


def hist_band_vs_flux(da):
       
    bands = [str(b.values) for b in da['band']]

    source = ColumnDataSource(
        data={'band': bands, 'flux': da.values})

    p = figure(x_range=bands)
    p.vbar(source=source, x='band', top='flux', width=.9)

    p.yaxis.axis_label = 'flux (W m-2)'
    p.xaxis.axis_label = 'spectral band'
    
    return p


def script_fluxerror():
    ds_crd = crd_data_atm(lblnew_params_atm())['flux']
    ds_clirad = clirad_data_atm(clirad_params_atm())['flux']
    
    ip, varname = 0, 'flug'
    da = (ds_crd - ds_clirad).isel(pressure=ip)[varname]
    p_toa = hist_band_vs_flux(da)

    ip, varname = -1, 'fldg'
    da = (ds_crd - ds_clirad).isel(pressure=ip)[varname]
    p_sfc = hist_band_vs_flux(da) 

    everything = gridplot(p_toa, p_sfc, ncols=2, 
                          plot_width=300, plot_height=300)
    
    show(everything)
    

In [176]:
script_atm_makeup()

clirad band,1,2,3,4,5,6,7,8,9,10,11
molecule,,,,,,,,,,,
h2o,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro,atmpro
co2,0,0,0.0004,0.0004,0.0004,0.0004,0.0004,0,0,0,0.0004
n2o,0,0,3.2e-07,0,0,0,0,0,3.2e-07,0,0
o3,0,0,0,0,0,0,atmpro,0,0,0,0
ch4,0,0,0,0,0,0,0,0,1.8e-06,0,0


Table. The makeup of the atmosphere.


In [177]:
script_cool()

Figure. Cooling rate.





Figure. Cooling rate difference.


In [178]:
script_fluxerror()

In [179]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')